## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [28]:
# Load up store_income_data.csv

import pandas as pd
import numpy as np

income_df = pd.read_csv("store_income_data_task.csv")

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [29]:
# Look all unique values, convert comumn to lowercase and remove any trailing white spaces

countries = income_df['country'].unique()
print(f"There are {len(countries)} unique countries before basic cleaning")
#print(countries)
# Convert to lower case
income_df['country'] = income_df['country'].str.lower()
# Remove trailing white spaces
income_df['country'] = income_df['country'].str.strip()
# Fill na too prevnet nan and null value
income_df.country.fillna('others', inplace=True)

countries = income_df['country'].unique()
print(f"There are {len(countries)} unique countries after basic cleaning")
#print(countries)

There are 77 unique countries before basic cleaning
There are 37 unique countries after basic cleaning


C:\Users\User\AppData\Local\Temp\ipykernel_52968\2698605234.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  income_df.country.fillna('others', inplace=True)


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [31]:
# Try to eliminate all variations and remain only three countries: South Africa, United Kingdom and United States

# Import Libraries for fuzzy word processing
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# Set seed for reproducibility
np.random.seed(0)

# Define function to use fuzzwuzzy to process the string with over 90% mark matching
def replace_country_name(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    #print(f"Checking:[{string_to_match}]")
    #print(matches)
    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    #print(f"Replacing [{close_matches}]")
    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print(f"Replacing[{string_to_match}] are done with ratio({min_ratio})!")

# List unique entry before replace
countries = income_df['country'].unique()
print("------------------------------------------------------------------")
print(f"There are {len(countries)} unique countries before fuzzy checking")

# Replace united kingdom
replace_country_name(df=income_df, column='country', string_to_match="united kingdom", min_ratio=90)
# Replace britian
replace_country_name(df=income_df, column='country', string_to_match="britain", min_ratio=90)
# Replace england
replace_country_name(df=income_df, column='country', string_to_match="england", min_ratio=90)
# Replace united states
replace_country_name(df=income_df, column='country', string_to_match="united states" ,min_ratio=90)
# Replace united states of america
replace_country_name(df=income_df, column='country', string_to_match="united states of america" ,min_ratio=90)
# Replace america
replace_country_name(df=income_df, column='country', string_to_match="america" ,min_ratio=90)
# Replace south africa
replace_country_name(df=income_df, column='country', string_to_match="south africa", min_ratio=75)
# Replace sa
replace_country_name(df=income_df, column='country', string_to_match="sa" ,min_ratio=40)
# Replace uk
replace_country_name(df=income_df, column='country', string_to_match="uk", min_ratio=40)

# List unique entry after replace
countries = income_df['country'].unique()
print(f"There are {len(countries)} unique countries after fuzzy checking")
print("------------------------------------------------------------------")
print(countries)

#Manual Replacing with same counntry with different naming
income_df.replace('uk', 'United Kingdom', inplace=True)
income_df.replace('britain', 'United Kingdom', inplace=True)
income_df.replace('england', 'United Kingdom', inplace=True)
income_df.replace('united kingdom', 'United Kingdom', inplace=True)
income_df.replace('united states', 'United States of America', inplace=True)
income_df.replace('united states of america', 'United States of America', inplace=True)
income_df.replace('america', 'United States of America', inplace=True)
income_df.replace('south africa', 'South Africa', inplace=True)
income_df.replace('sa', 'South Africa', inplace=True)
income_df.replace('/', 'Others', inplace=True)
income_df.replace('others', 'Others', inplace=True)
income_df.replace('.', 'Others', inplace=True)
income_df.replace('', 'Others', inplace=True)

countries = income_df['country'].unique()
print(f"There are {len(countries)} unique countries after manual renaming")
print(countries)



------------------------------------------------------------------
There are 4 unique countries before fuzzy checking
Replacing[united kingdom] are done with ratio(90)!
Replacing[britain] are done with ratio(90)!
Replacing[england] are done with ratio(90)!
Replacing[united states] are done with ratio(90)!
Replacing[united states of america] are done with ratio(90)!
Replacing[america] are done with ratio(90)!
Replacing[south africa] are done with ratio(75)!
Replacing[sa] are done with ratio(40)!
Replacing[uk] are done with ratio(40)!
There are 4 unique countries after fuzzy checking
------------------------------------------------------------------
['united states of america' 'united kingdom' 'south africa' 'Others']
There are 4 unique countries after manual renaming
['United States of America' 'United Kingdom' 'South Africa' 'Others']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [32]:
# Import modules
import datetime

#Convert date_measured into data value
income_df['date_measured'] = pd.to_datetime(income_df['date_measured'], format='%d-%m-%Y')
# Set today temporary into days ago for calculating next
income_df['days_ago'] = pd.to_datetime(datetime.date.today(), format='%d-%m-%Y')
income_df['days_ago'] = income_df['days_ago'] - income_df['date_measured']

# Show sample result
income_df.head()


,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,2006-02-04,United States of America,6652 days
1,2,Nordson Corporation,NaN,Tools,$41744177.01,2006-01-04,United Kingdom,6683 days
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,2003-09-12,United States of America,7528 days
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,2006-05-08,United Kingdom,6559 days
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,1973-01-21,United Kingdom,18719 days
